In [1]:
import numpy as np
import pandas
import re
import math
import random
import nltk
import time
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TweetTokenizer
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from time import time

# SVM

## Binary Classification

In [2]:
from cvxopt import matrix
from cvxopt import solvers

### Loading the SVM data 

In [3]:
d = 2
svm_train_data = pandas.read_csv('./data/fashion_mnist/train.csv').values
svm_test_data = pandas.read_csv('./data/fashion_mnist/test.csv').values
svm_validation_data = pandas.read_csv('./data/fashion_mnist/val.csv').values

### Linear

In [4]:
# Required data
svm_train_data_r = svm_train_data[(svm_train_data[:, -1] == (d + 1) % 10) + (svm_train_data[:, -1] == d), :]
svm_test_data_r = svm_test_data[(svm_test_data[:, -1] == (d + 1) % 10) + (svm_test_data[:, -1] == d), :]
svm_validation_data_r = svm_validation_data[(svm_validation_data[:, -1] == (d + 1) % 10) + (svm_validation_data[:, -1] == d), :]
svm_train_data_r.shape

(4500, 785)

In [5]:
# Reading y's
y_train = svm_train_data_r[:, -1]
y_train[y_train == d] = -1
y_train[y_train == (d + 1) % 10] = 1

y_test = svm_test_data_r[:, -1]
y_test[y_test == d] = -1
y_test[y_test == (d + 1) % 10] = 1

y_val = svm_validation_data_r[:, -1]
y_val[y_val == d] = -1
y_val[y_val == (d + 1) % 10] = 1

In [6]:
# Reading x's and normalising
x_train = (np.delete(svm_train_data_r, -1, axis = 1) / 255).T
x_test = (np.delete(svm_test_data_r, -1, axis = 1) / 255).T
x_val = (np.delete(svm_validation_data_r, -1, axis = 1) / 255).T

# Noise
C = 1.0

In [7]:
# Calculating P matrix
y_diag = y_train.reshape((-1, 1))
p_matrix = np.matmul(x_train.T, x_train)
p_matrix = y_diag * p_matrix
y_diag = y_diag.T
p_matrix = p_matrix * y_diag
p_matrix = matrix(p_matrix)

In [8]:
# Calculating q matrix
q_matrix = -matrix(np.ones_like(y_train))

In [9]:
# Calculating A and b matrices
A_matrix = matrix(y_diag) 
b_matrix = matrix([0.0])

In [10]:
# Calculating G and h matrices
G_1 = np.identity(np.shape(y_train)[0])
h_1 = C * np.ones_like(y_train)
G_2 = -G_1
h_2 = 0 * h_1

G_matrix = matrix(np.vstack((G_1, G_2)))
h_matrix = matrix(np.append(h_1, h_2))

In [ ]:
%time
# Solving the problem
sol = solvers.qp(p_matrix, q_matrix, G_matrix, h_matrix, A_matrix, b_matrix)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
     pcost       dcost       gap    pres   dres
 0: -5.2396e+02 -9.3983e+03  5e+04  3e+00  3e-12
 1: -3.3568e+02 -5.4793e+03  1e+04  5e-01  3e-12
 2: -2.0714e+02 -1.8273e+03  3e+03  1e-01  2e-12
 3: -1.4999e+02 -9.1602e+02  1e+03  5e-02  1e-12
 4: -1.1131e+02 -4.9405e+02  6e+02  2e-02  1e-12
 5: -9.0544e+01 -3.1115e+02  3e+02  8e-03  9e-13
 6: -8.5666e+01 -1.4689e+02  7e+01  6e-04  1e-12
 7: -9.2854e+01 -1.1845e+02  3e+01  2e-14  1e-12


In [ ]:
# Calculating alpha
alpha = np.array(sol['x'])
print("Number of support vectors are:", np.where(alpha > 1e-5)[0].shape[0])

In [ ]:
# Calculating W
y_diag = (y_train.reshape((-1, 1))).T
w = (x_train * y_diag) @ alpha

In [ ]:
# Calculating b
x_temp_0 = svm_train_data[(svm_train_data[:, -1] == d), :]
x_temp_0 = (np.delete(x_temp_0, -1, axis = 1) / 255).T
x_temp_1 = svm_train_data[(svm_train_data[:, -1] == (d + 1) % 10), :]
x_temp_1 = (np.delete(x_temp_1, -1, axis = 1) / 255).T
x_temp_0.shape

In [ ]:
maximum = np.max(w.T @ x_temp_0)
minimum = np.min(w.T @ x_temp_1)

In [ ]:
b = - (maximum + minimum) / 2.0

In [ ]:
# Predicting on test data
pred_test = (w.T @ x_test) + b
pred_test[pred_test >= 0] = 1
pred_test[pred_test < 0] = -1
count = 0
total = pred_test.shape[1]
for (i, p) in enumerate(pred_test[0, :]):
    if p == y_test[i]:
        count += 1
print("Accuracy is {}".format(count * 100 / total))

In [ ]:
# Predicting on validation data
pred_val = (w.T @ x_val) + b
pred_val[pred_val >= 0] = 1
pred_val[pred_val < 0] = -1
count = 0
total = pred_val.shape[1]
for (i, p) in enumerate(pred_val[0, :]):
    if p == y_val[i]:
        count += 1
print("Accuracy is {}".format(count * 100 / total))

### Gaussian

In [ ]:
from scipy.spatial.distance import pdist, cdist, squareform

In [ ]:
# Calculating Gaussian
gamma = 0.05
m = y_train.shape[0]
pairwise_dists = squareform(pdist(x_train.T, 'euclidean'))
k_matrix = np.exp(- gamma * pairwise_dists ** 2)

In [ ]:
# Changing P_matrix
y_diag = y_train.reshape((-1, 1))
p_matrix = k_matrix
p_matrix = y_diag * p_matrix
y_diag = y_diag.T
p_matrix = p_matrix * y_diag
p_matrix = matrix(p_matrix)

In [ ]:
# Solving the problem
sol = solvers.qp(p_matrix, q_matrix, G_matrix, h_matrix, A_matrix, b_matrix)

In [ ]:
# Calculating alpha
alpha = np.array(sol['x'])
print("Number of support vectors are: ", np.where(alpha > 1e-5)[0].shape[0])

In [ ]:
# Returning wTx
# pairwise_dists_2 = (cdist(x_train.T, x_test.T))
# k_matrix_test = np.exp(- gamma * pairwise_dists_2 ** 2)
# def get_w_t_x(x_j):
#     x_temp = np.exp(-gamma * np.linalg.norm(x_train - x_j[:, None], axis = 0) ** 2)
#     return np.sum(alpha[:, 0] * y_train * x_temp)

In [ ]:
# Calculating B
x_temp_0 = svm_train_data[(svm_train_data[:, -1] == d), :]
x_temp_0 = (np.delete(x_temp_0, -1, axis = 1) / 255).T
x_temp_1 = svm_train_data[(svm_train_data[:, -1] == (d + 1) % 10), :]
x_temp_1 = (np.delete(x_temp_1, -1, axis = 1) / 255).T
x_temp_0.shape

In [ ]:
x_list_0 = np.sum(y_train[:, None] * alpha * np.exp(-gamma * cdist(x_train.T, x_temp_0.T) ** 2), axis = 0)
x_list_1 = np.sum(y_train[:, None] * alpha * np.exp(-gamma * cdist(x_train.T, x_temp_1.T) ** 2), axis = 0)
# print(x_list_0.shape)
# x_list_0 = np.apply_along_axis(get_w_t_x, 0, x_temp_0)
# x_list_1 = np.apply_along_axis(get_w_t_x, 0, x_temp_1)

In [ ]:
b = -(np.max(x_list_0) + np.min(x_list_1)) / 2.0

In [ ]:
# Predicting on test data
t0 = time()
pairwise_dists_2 = (cdist(x_train.T, x_test.T))
k_matrix_test = y_train[:, None] * alpha * np.exp(- gamma * pairwise_dists_2 ** 2)
pred_test = np.sum(k_matrix_test, axis = 0) + b
pred_test[pred_test >= 0] = 1
pred_test[pred_test < 0] = -1
count = 0
total = pred_test.shape[0]
for (i, p) in enumerate(pred_test):
    if p == y_test[i]:
        count += 1
print("Accuracy is {}".format(count * 100 / total))
print(f"Prediction time (test): {round(time()-t0, 3)}s")

In [ ]:
# Predicting on validation data
t0 = time()
pairwise_dists_2 = (cdist(x_train.T, x_val.T))
k_matrix_val = y_train[:, None] * alpha * np.exp(- gamma * pairwise_dists_2 ** 2)
pred_val = np.sum(k_matrix_val, axis = 0) + b
pred_val[pred_val >= 0] = 1
pred_val[pred_val < 0] = -1
count = 0
total = pred_val.shape[0]
for (i, p) in enumerate(pred_val):
    if p == y_val[i]:
        count += 1
print("Accuracy is {}".format(count * 100 / total))
print(f"Prediction time (Validation): {round(time()-t0, 3)}s")

### Sklearn

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
# Linear 
# Create a svm Classifier
clf = SVC(kernel='linear', C=C) # Linear Kernel

# Train the model using the training sets
clf.fit(x_train.T, y_train)

# Predict the response for test dataset
y_pred = clf.predict(x_test.T)

# Predicting accuracy
print("Linear Accuracy(test):", metrics.accuracy_score(y_test, y_pred))

# Predict the response for validation dataset
y_pred = clf.predict(x_val.T)

# Predicting accuracy
print("Linear Accuracy(validation):", metrics.accuracy_score(y_val, y_pred))
print("Number of support vectors:", np.sum(clf.n_support_))

In [ ]:
# Gaussian
# Create a svm Classifier
clf = SVC(kernel='rbf', gamma=gamma, C=C) # Gaussian Kernel

# Train the model using the training sets
clf.fit(x_train.T, y_train)

# Predict the response for test dataset
y_pred = clf.predict(x_test.T)

# Predicting accuracy
print("Gaussian Accuracy(test):", metrics.accuracy_score(y_test, y_pred))

# Predict the response for validation dataset
y_pred = clf.predict(x_val.T)

# Predicting accuracy
print("Gaussian Accuracy(validation):", metrics.accuracy_score(y_val, y_pred))
print("Number of support vectors:", np.sum(clf.n_support_))

## Multiclass Classification

### Gaussian

In [ ]:
# Reading training data
X_train = {}
Y_train = {}
for i in range(10):
    svm_train_data_r = svm_train_data[(svm_train_data[:, -1] == i), :]
    Y_train[i] = svm_train_data_r[:, -1]
    X_train[i] = (np.delete(svm_train_data_r, -1, axis = 1) / 255).T

# Reading test data
X_test = {}
Y_test = {}
for i in range(10):
    svm_test_data_r = svm_test_data[(svm_test_data[:, -1] == i), :]
    Y_test[i] = svm_test_data_r[:, -1]
    X_test[i] = (np.delete(svm_test_data_r, -1, axis = 1) / 255).T

In [ ]:
# Calculating P matrices


### Sklearn